# Microsoft Malware Challenge

## EDA 2 & Baseline model

## For CSCE 633 Machine Learning, Spring 2019, Course project
### Team: MARTHA
### Author: Rose Lin

This is the second notebook in the series. We have already performed a quick scan among all the variables in our *EDA1* notebook. Here we would like to check again in closer details and address some of the issues raised in the first notebook.

~Ongoing efforts, your feedback is appreciated!~

### RECAP

In EDA 1, we ended up concluding that the following 66 features can be kept for our model:

```
['ProductName' 'EngineVersion' 'AppVersion' 'AvSigVersion' 'IsBeta'
 'RtpStateBitfield' 'AVProductStatesIdentifier' 'AVProductsInstalled'
 'AVProductsEnabled' 'HasTpm' 'CountryIdentifier' 'CityIdentifier'
 'OrganizationIdentifier' 'GeoNameIdentifier'
 'LocaleEnglishNameIdentifier' 'Processor' 'OsVer' 'OsBuild' 'OsSuite'
 'OsPlatformSubRelease' 'OsBuildLab' 'IsProtected' 'AutoSampleOptIn'
 'SMode' 'IeVerIdentifier' 'SmartScreen' 'Firewall' 'UacLuaenable'
 'Census_MDC2FormFactor' 'Census_DeviceFamily' 'Census_OEMNameIdentifier'
 'Census_OEMModelIdentifier' 'Census_ProcessorCoreCount'
 'Census_ProcessorModelIdentifier' 'Census_PrimaryDiskTotalCapacity'
 'Census_PrimaryDiskTypeName' 'Census_SystemVolumeTotalCapacity'
 'Census_HasOpticalDiskDrive' 'Census_TotalPhysicalRAM'
 'Census_ChassisTypeName'
 'Census_InternalPrimaryDiagonalDisplaySizeInInches'
 'Census_InternalPrimaryDisplayResolutionHorizontal'
 'Census_PowerPlatformRoleName' 'Census_InternalBatteryNumberOfCharges'
 'Census_OSVersion' 'Census_OSBranch' 'Census_OSBuildRevision'
 'Census_OSEdition' 'Census_OSInstallTypeName'
 'Census_OSInstallLanguageIdentifier' 'Census_OSWUAutoUpdateOptionsName'
 'Census_IsPortableOperatingSystem' 'Census_GenuineStateName'
 'Census_ActivationChannel' 'Census_IsFlightsDisabled' 'Census_FlightRing'
 'Census_FirmwareManufacturerIdentifier'
 'Census_FirmwareVersionIdentifier' 'Census_IsSecureBootEnabled'
 'Census_IsVirtualDevice' 'Census_IsTouchEnabled' 'Census_IsPenCapable'
 'Census_IsAlwaysOnAlwaysConnectedCapable' 'Wdft_IsGamer'
 'Wdft_RegionIdentifier' 'HasDetections']
```

But is this correct? 

We also need to consider the following issues:

1. NANs - how to better handle those missing values? (Need a throughout investigation)
2. Skewed variables - how to fix their skewness?
3. More feature selection/combination - can we NOT drop features but map them into another space?
4. Algorithm improvement - the **real** meat and butter.

So this notebook would attempt to address the first 3, and build a baseline SVM model to check its performance.

In [0]:
# load the data from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(r"/content/gdrive/My Drive/Coding experiment/MARTHA/data"))

# Any results you write to the current directory are saved as output.

In [0]:
# from https://www.kaggle.com/theoviel/load-the-totality-of-the-data
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [0]:
# load the data
train = pd.read_csv("/content/gdrive/My Drive/Coding experiment/MARTHA/data/train.csv",dtype = dtypes)
ID_col = train['MachineIdentifier']
train.drop('MachineIdentifier', axis=1, inplace=True)
# take a look at the head
train.head()

Note: if using LightGBM we don't have to do feature selection (auto selected, based on decision trees)